<a href="https://colab.research.google.com/github/LuLu1-1/AB-testing/blob/main/Website_design_A_B_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive/') 

Mounted at /content/gdrive/


In [43]:
import pandas as pd
import numpy as np
import math
import statsmodels.stats.api as sms
import scipy.stats as st
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [37]:
data = pd.read_csv('/content/gdrive/MyDrive/small dataset/ab_data.csv')
print(data.shape)
data.head()

(294478, 5)


,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


# data cleaning

In [16]:
data.pivot_table(index = 'group',columns='landing_page',values = 'user_id',aggfunc='count')

landing_page,new_page,old_page
group,,
control,1928,145274
treatment,145311,1965


In [33]:
session_counts = data['user_id'].value_counts(ascending=False)
multi_users = session_counts[session_counts>1].count()
multi_users

3894

In [34]:
# users = session_counts[session_counts < 2].index
# data = data[data['user_id'].isin(users)]

In [38]:
filter1 = (data["group"] == "control") & (data["landing_page"] == "new_page")
index_to_drop1 = data[filter1].index
data = data.drop(index_to_drop1)
filter2 = (data["group"] == "treatment") & (data["landing_page"] == "old_page")
index_to_drop2 = data[filter2].index
data = data.drop(index_to_drop2)
print(data.shape)
data["group"].value_counts()

(290585, 5)


treatment    145311
control      145274
Name: group, dtype: int64

In [39]:
print(data["user_id"].count())
print(data["user_id"].nunique())

290585
290584


In [40]:
data.drop_duplicates(subset ='user_id',keep ='first',inplace = True)

# a/b test

In [11]:
control = (data["group"] == "control")
conversions_control = data["converted"][control].sum()
total_users_control = data["converted"][control].count()

treat = (data["group"] == "treatment")
conversions_treatment = data["converted"][treat].sum()
total_users_treatment = data["converted"][treat].count()

print("samples rate in control users and treat users : ", 
    round(total_users_control / data["converted"].count() * 100, 2), "% ",
    round((total_users_treatment / data["converted"].count()) * 100, 2), "%")
#count number of users who converted in each group
print("Number of control users who converted on old page: ", conversions_control)
print("Percentage of control users who converted: ", round((conversions_control / total_users_control) * 100, 2), "%")
print("Number of treatment users who converted on new page: ", conversions_treatment)
print("Percentage of treatment users who converted: ", round((conversions_treatment/ total_users_treatment) * 100, 2), "%")

samples rate in control users and treat users :  49.99 %  50.01 %
Number of control users who converted on old page:  17489
Percentage of control users who converted:  12.04 %
Number of treatment users who converted on new page:  17264
Percentage of treatment users who converted:  11.88 %


 check sample size

In [13]:
baseline_rate = conversions_control / total_users_control
practical_significance = 0.01 #user defined
confidence_level = 0.05 #user defined, for a 95% confidence interval
sensitivity = 0.8 #user defined

effect_size = sms.proportion_effectsize(baseline_rate, baseline_rate + practical_significance)
sample_size = sms.NormalIndPower().solve_power(effect_size = effect_size, power = sensitivity, 
                          alpha = confidence_level, ratio=1)
print("Required sample size: ", round(sample_size), " per group")

Required sample size:  17209  per group


In [ ]:
# random sampling
# control_sample = data[data['group'] == 'control'].sample(n=sample_size, random_state=22)
# treatment_sample = data[data['group'] == 'treatment'].sample(n=sample_size, random_state=22)
# ab_test = pd.concat([control_sample, treatment_sample], axis=0)
# ab_test.reset_index(drop=True, inplace=True)
# ab_test.groupby('group')['landing_page'].value_counts()
# conversion_rates = ab_test.groupby('group')['converted'].mean().to_frame()
# control_results = ab_test[ab_test['group'] == 'control']['converted']     #获取对照组是否转化的数据
# treatment_results = ab_test[ab_test['group'] == 'treatment']['converted'] #获取实验组是否转化的数据
# n_con = control_results.count() # 获取对照组人数
# n_treat = treatment_results.count()  # 获取实验组人数
# successes = [control_results.sum(), treatment_results.sum()] # 获取实验组和对照组成功转化的人数
# nobs = [n_con, n_treat]
# z_stat, pval = proportions_ztest(successes, nobs=nobs)  #计算P值
# (lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05) #计算置信区间
# print(f'z statistic: {z_stat:.2f}')
# print(f'p-value: {pval:.3f}')
# print(f'ci 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
# print(f'ci 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

In [42]:
prob_pooled = (conversions_control + conversions_treatment) / (total_users_control + total_users_treatment)
se_pooled = math.sqrt(prob_pooled * (1 - prob_pooled) * (1 / total_users_control + 1 / total_users_treatment))
z_score = st.norm.ppf(1 - confidence_level / 2)
margin_of_error = se_pooled * z_score

#Calculate dhat, the estimated difference between probability of conversions in the experiment and control groups
d_hat = (conversions_treatment / total_users_treatment) - (conversions_control / total_users_control)

#Test if we can reject the null hypothesis
lower_bound = d_hat - margin_of_error
upper_bound = d_hat + margin_of_error

if practical_significance < lower_bound:
    print("Reject null hypothesis")
else: 
    print("Accept the null hypothesis")
    
print("The lower bound of the confidence interval is ", round(lower_bound * 100, 2), "%")
print("The upper bound of the confidence interval is ", round(upper_bound * 100, 2), "%")

Accept the null hypothesis
The lower bound of the confidence interval is  -0.39 %
The upper bound of the confidence interval is  0.08 %
